In [1]:
import sys

sys.path.append("/home/maximilien/work/pixano/")
sys.path.append("/home/maximilien/work/lib/bop_toolkit")

dir = "/home/maximilien/work/bop_data/"
coco = "/home/maximilien/work/adapt-2023_with_gt/test/coco_gt.json"

In [2]:
import json
from pathlib import Path

import bop_toolkit_lib.dataset.bop_webdataset as btk
import lance
import pyarrow as pa
import webdataset as wds
from lance import LanceDataset
from PIL import Image as pilImage

from pixano.data import DatasetInfo
from pixano.data.importers import DataImporter
from pixano.types import (
    BBox,
    Camera,
    CameraType,
    CompressedRLE,
    CompressedRLEType,
    DepthImage,
    DepthImageType,
    Fields,
    GtInfo,
    GtInfoType,
    Image,
    ImageType,
    Pose,
    PoseType,
)
from pixano.utils import image_to_binary, image_to_thumbnail

In [3]:
class BOPImporter(DataImporter):

    def __init__(
        self,
        shard_split:dict[str, list[str]],
        info:DatasetInfo,
        target_dir: Path,
    ):
        
        self.shard_split = shard_split
        self.info = info
        self.target_dir = target_dir

    @property
    def fields(self) -> Fields:
        return Fields.from_string_dict(self.info.fields)

    def create_json(self):
        """Create dataset spec.json"""

        # Read dataset
        #"dataset = ds.dataset(self.target_dir + "/db", partitioning=self.partitioning)

        # Check number of rows in the created dataset
        #self.info.num_elements = dataset.count_rows()

        # Create spec.json
        with open(self.target_dir + "/spec.json", "w") as f:
            json.dump(vars(self.info), f, indent=4)


    def fields(self):
        return self.features.to_fields()
    
    def schema(self):
        return pa.schema(self.fields())
    
    def get_row(self):
        
        #split dataset
        for split, shard_list in self.shard_split.items():
            print(split)

            _wds_pipeline = wds.DataPipeline(
                wds.SimpleShardList(shard_list),
                wds.tarfile_to_samples()
            )
            
            #extract row of each split
            for n, row in enumerate(_wds_pipeline):

                if True:
                    sample = btk.decode_sample(
                        row,
                        decode_camera=True,
                        decode_rgb=True,
                        decode_gray=False,
                        decode_depth=True,
                        decode_gt=True,
                        decode_gt_info=True,
                        decode_mask_visib=False,
                        decode_mask=False,
                        rgb_suffix='.png'
                    )

                    #id
                    id = row["__key__"]

                    scene, image = id.split('_')
                    coco_json_path = f'/home/maximilien/work/adapt-2023_with_gt/{split}/{scene}/scene_gt_coco.json'

                    #rgb
                    im_pil = pilImage.fromarray(sample['im_rgb'])

                    im_pil = image_to_binary(im_pil, format='JPEG')
                    
                    preview = image_to_thumbnail(im_pil)

                    rgb = Image(f"", im_pil, preview)
                    rgbs = ImageType.Array.from_list([rgb])
                    #dept
                    depths = DepthImageType.Array.from_list([DepthImage(depth_map=sample["im_depth"], shape=sample["im_depth"].shape)])
                    #camera
                    cameras = CameraType.Array.from_list([Camera.from_dict(sample['camera'])])


                #Objects
                    nb_object = len(sample['gt'])
                    #category
                    category_id = [sample['gt'][i]['object_id'] for i in range(nb_object)]
                    category_id_arr = pa.array([category_id])

                    #pose
                    gt = [Pose(sample['gt'][i]['cam_R_m2c'].flatten(), sample['gt'][i]['cam_t_m2c'].flatten()) for i in range(nb_object)]
                    gt_arr = PoseType.Array.from_lists([gt])

                    #gt_info
                    gt_infos = [
                        GtInfo.from_dict(
                            {
                                **sample["gt_info"][i],
                                "bbox_obj": BBox.from_xywh(sample["gt_info"][i]["bbox_obj"]),
                                "bbox_visib": BBox.from_xywh(sample["gt_info"][i]["bbox_visib"]),
                            }
                        )
                        for i in range(nb_object)
                    ]
                    gt_infos_arr = GtInfoType.Array.from_lists([gt_infos])

                    #objects_ids and masks
                    with open(coco_json_path, 'r') as f:
                        data = json.load(f)
        
                    object_ids = []
                    masks = []
                    for ann in data['annotations']:
                        #check if same image key, then annotations are in same order as other object's attribute in coco.json
                        if '000' + ann['image_id'] == id.replace('_','-'):
                            object_ids.append(ann['id'])
                            masks.append(CompressedRLE.from_urle(ann['segmentation'], ann['segmentation']['size'][0], ann['segmentation']['size'][1]))

                    masks_arr = CompressedRLEType.Array.from_lists([masks])
                    object_ids_arr = pa.array([object_ids])

                #Struct array
                    struct_arr = pa.StructArray.from_arrays(
                        [
                            pa.array([id]),
                            rgbs,
                            depths,
                            cameras,
                            category_id_arr,
                            object_ids_arr,
                            masks_arr,
                            gt_arr,
                            gt_infos_arr,
                            pa.array([split])
                        ],
                        fields=self.fields()
                    )

                    yield pa.RecordBatch.from_struct_array(struct_arr)


    def import_dataset(self, max_rows_per_file: int = 1024*1024, max_rows_per_group:int = 1024) -> LanceDataset:
        """Import dataset to Pixano format

        Args:
            batch_size (int, optional): Number of rows per file. Defaults to 2048.
        """

        reader = pa.RecordBatchReader.from_batches(self.schema(), self.get_row())
        ds = lance.write_dataset(reader, self.target_dir)

        self.info.num_elements = ds.count_rows()

        # Create spec.json
        self.create_json()
        
        return ds


In [4]:
import os

shard_test_dir = "/home/maximilien/work/adapt-2023_with_gt/shard/shard_test/"
shard_test_list = [os.path.join(shard_test_dir, shard) for shard in os.listdir(shard_test_dir) if shard.endswith(".tar")]

shard_validation_dir = "/home/maximilien/work/adapt-2023_with_gt/shard/shard_validation/"
shard_validation_list = [os.path.join(shard_validation_dir, shard) for shard in os.listdir(shard_validation_dir) if shard.endswith(".tar")]

shard_split = {'test':shard_test_list, 'val': shard_validation_list}

In [5]:
field_dict = {
    'id': 'str',
    'rgb': 'Image',
    'depth': 'DepthImage',
    'camera': 'Camera',
    'category_id': '[int]',
    'objects_id': '[str]',
    'masks': '[CompressedRLE]',
    'gt': '[Pose]',
    'gt_info':'[GtInfo]',
    'split':'str'
}

bop_info = DatasetInfo(id="0", name="Bop", description="Bop dataset", fields=field_dict)

bop_importer = BOPImporter(shard_split, bop_info, dir + '/bop_ds.lance')


In [6]:
bop_ds = bop_importer.import_dataset()

test


/home/maximilien/miniconda3/envs/pixano/lib/python3.11/site-packages/imageio/plugins/pillow.py:297: UserWarning: Loading 16-bit (uint16) PNG as int32 due to limitations in pillow's PNG decoder. This will be fixed in a future version of pillow which will make this warning dissapear.
  warnings.warn(


val
